In [ ]:
import os
import json
from dotenv import dotenv_values
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet
from azure.identity import DefaultAzureCredential

In [ ]:
env = dotenv_values(".env")

In [ ]:
def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json


In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=env["FOUNDRY_CONN"]
)

functions = FunctionTool([fetch_weather])
toolset = ToolSet()
toolset.add(functions)

agent = project_client.agents.create_agent(
    model="gpt-4o", name="weather-agent", instructions="You are a weather bot. Use the provided functions to help answer questions.", toolset=toolset
)

In [ ]:
thread = project_client.agents.create_thread()

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Do I need an umbrella in New York today?",
)


In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")